# <div style='color:#ff77e0;font-weight:800;'>用选好超参数的模型去训练</div>


In [1]:


from nes_py.wrappers import JoypadSpace
import gym_super_mario_bros
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT
import time
from matplotlib import pyplot as plt
from gym.wrappers import GrayScaleObservation
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.vec_env import VecFrameStack
import os
from stable_baselines3 import PPO

from stable_baselines3.common.results_plotter import load_results, ts2xy
import numpy as np
from stable_baselines3.common.callbacks import BaseCallback



In [2]:
env = gym_super_mario_bros.make('SuperMarioBros-v0')
env = JoypadSpace(env, SIMPLE_MOVEMENT)

monitor_dir = r'./monitor_log/'
os.makedirs(monitor_dir,exist_ok=True)
env = Monitor(env,monitor_dir)

env = GrayScaleObservation(env,keep_dim=True)
env = DummyVecEnv([lambda: env])
env = VecFrameStack(env,4,channels_order='last')


In [8]:
bast_params={
    'n_steps': 7168,
    'gamma': 0.8692871366327747,
    'learning_rate': 6.442559213980066e-05,
    'clip_range': 0.31688308594665404,
    'gae_lambda': 0.8710254680014865
}


In [13]:

# ! 设置对应的参数

tensorboard_log = r'./tensorboard_log/'

model = PPO("CnnPolicy", env, verbose=1,
            tensorboard_log = tensorboard_log,
            **bast_params
            )

Using cuda device
Wrapping the env in a VecTransposeImage.


In [14]:

save_model_dir  = r'./best_model/model_1500000.zip'

model.set_parameters(save_model_dir)

In [4]:
class SaveOnBestTrainingRewardCallback(BaseCallback):
    """
    Callback for saving a model (the check is done every ``check_freq`` steps)
    based on the training reward (in practice, we recommend using ``EvalCallback``).

    :param check_freq: (int)
    :param log_dir: (str) Path to the folder where the model will be saved.
      It must contains the file created by the ``Monitor`` wrapper.
    :param verbose: (int)
    """

    def __init__(self, check_freq, save_model_dir, verbose=1):
        super(SaveOnBestTrainingRewardCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = os.path.join(save_model_dir, 'best_model/')
        self.best_mean_reward = -np.inf

    # def _init_callback(self) -> None:
    def _init_callback(self):
        # Create folder if needed
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    # def _on_step(self) -> bool:
    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            print('self.n_calls: ',self.n_calls)
            model_path1 = os.path.join(self.save_path, 'model_{}'.format(self.n_calls))
            self.model.save(model_path1)

        return True

In [15]:
save_model_dir = r'./'
callback1 = SaveOnBestTrainingRewardCallback(200000, save_model_dir)

model.learn(total_timesteps=3000000,callback=callback1,reset_num_timesteps=False)
# model.save("mario_model")

Logging to ./tensorboard_log/PPO_3
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1.38e+03 |
|    ep_rew_mean     | 2.12e+03 |
| time/              |          |
|    fps             | 200      |
|    iterations      | 1        |
|    time_elapsed    | 35       |
|    total_timesteps | 7168     |
---------------------------------
---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 2.14e+03  |
|    ep_rew_mean          | 2.27e+03  |
| time/                   |           |
|    fps                  | 68        |
|    iterations           | 2         |
|    time_elapsed         | 208       |
|    total_timesteps      | 14336     |
| train/                  |           |
|    approx_kl            | 0.3892345 |
|    clip_fraction        | 0.227     |
|    clip_range           | 0.317     |
|    entropy_loss         | -0.46     |
|    explained_variance   | 0.967     |
|    learning_rate       

KeyboardInterrupt: 